I will want to beat the baseline score of $R^2=0.24$ and $RMSE=1,881$.

In [1]:
from __future__ import division
import pandas as pd

In [2]:
import warnings
import seaborn as sns

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
from zipcode_mapping import zipcode_mapping

In [4]:
from scipy.sparse import csr_matrix

In [5]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips_and_descs_and_dropped_latlongs.pkl')
df.shape

(744372, 37)

In [6]:
df.head(1).T

,905223
CASE_ENQUIRY_ID,101001983804
OPEN_DT,2017-01-08 12:00:16
TARGET_DT,2017-01-10 08:30:00
CLOSED_DT,NaT
CASE_TITLE,Parking Enforcement
SUBJECT,Transportation - Traffic Division
REASON,Enforcement & Abandoned Vehicles
TYPE,Parking Enforcement
Department,BTDT
SubmittedPhoto,False


In [6]:
df.Property_Type = df.Property_Type.fillna('other')

In [8]:
df[['Property_Type', 'neighborhood_from_zip']].isnull().sum()

Property_Type            0
neighborhood_from_zip    0
dtype: int64

## Let's do other features in the original dataset first

In [9]:
old_df = df.copy()

In [10]:
df.head(1)#.loc[:, 'SubmittedPhoto':]

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,CASE_TITLE,SUBJECT,REASON,TYPE,Department,SubmittedPhoto,...,COMPLETION_TIME,school,housing,bedroom,value,rent,income,is_issue_unresolved,zipcode,neighborhood_from_zip
905223,101001983804,2017-01-08 12:00:16,2017-01-10 08:30:00,NaT,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,BTDT,False,...,NaN,20_bachelors,own,3,350000.0,2250,87500,True,2131.0,Roslindale


In [7]:
df = df[['TYPE', 'SubmittedPhoto', 'Property_Type', 'Source', 'zipcode', 'COMPLETION_TIME']]

In [8]:
df.isnull().sum()

TYPE                   0
SubmittedPhoto         0
Property_Type          0
Source                 0
zipcode                0
COMPLETION_TIME    65765
dtype: int64

In [9]:
df = df.dropna()

In [14]:
df.isnull().sum()

TYPE                     0
REASON                   0
Department               0
SubmittedPhoto           0
Property_Type            0
Source                   0
neighborhood_from_zip    0
zipcode                  0
COMPLETION_TIME          0
dtype: int64

In [10]:
df.shape

(678607, 6)

## Dummifying

In [ ]:
def dummify(df, column):
    # from Darren's linear regression slides
    print '{} is your baseline'.format(sorted(df[column].unique())[-1])
    dummy = pd.get_dummies(df[column]).rename(columns=lambda x: column+'_'+str(x)).iloc[:,0:len(df[column].unique())-1]
    df = df.drop(column,axis=1) #Why not inplace? because if we do inplace, it will affect the df directly
    return pd.concat([df,dummy],axis=1)

In [17]:
df1 = dummify(df, 'TYPE')
df2 = dummify(df1, 'REASON')
df1 = None
df3 = dummify(df2, 'Department')
df2 = None
df4 = dummify(df3, 'Property_Type')
df3 = None
df5 = dummify(df4, 'Source')
df4 = None
df6 = dummify(df5, 'neighborhood_from_zip')
df5 = None
df7 = dummify(df6, 'zipcode')

Zoning is your baseline
Weights and Measures is your baseline
PWDx is your baseline
other is your baseline
Twitter is your baseline
West Roxbury is your baseline
2467.0 is your baseline


In [11]:
def dummify_cols_and_baselines(df, cols):
    baseline_cols = []
    
    for i, column in enumerate(cols):
        baseline = sorted(df[column].unique())[-1]
        print baseline, 'is baseline', i, len(cols)
        baseline_cols += [baseline]
        dummy = pd.get_dummies(df[column]).rename(columns=lambda x: column+'_'+str(x)).iloc[:,0:len(df[column].unique())-1]
        df = df.drop(column, axis=1) #Why not inplace? because if we do inplace, it will affect the df directly
        df = pd.concat([df, dummy], axis=1)
        
    return df, baseline_cols

In [14]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'TYPE', u'Property_Type', u'Source'], dtype='object')

In [15]:
df7, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Zoning is baseline 0 3
other is baseline 1 3
Twitter is baseline 2 3


In [18]:
df7.shape

(678607, 213)

In [19]:
df7.head()

,SubmittedPhoto,zipcode,COMPLETION_TIME,TYPE_ADA,TYPE_Abandoned Bicycle,TYPE_Abandoned Building,TYPE_Abandoned Vehicles,TYPE_Alert Boston,TYPE_Animal Found,TYPE_Animal Generic Request,...,TYPE_Working Beyond Hours,TYPE_Yardwaste Asian Longhorned Beetle Affected Area,Property_Type_Address,Property_Type_Intersection,Source_Citizens Connect App,Source_City Worker App,Source_Constituent Call,Source_Employee Generated,Source_Maximo Integration,Source_Self Service
905425,True,2124.0,0.918333,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
905235,False,2128.0,2.325000,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
905379,False,2128.0,2.573333,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
905264,False,2124.0,1.030000,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
905447,False,2124.0,1.681944,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


## Let's run the model

In [20]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Splitting train/test 80/20:

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    df7.drop('COMPLETION_TIME', axis=1), 
    df7.COMPLETION_TIME, 
    test_size=0.2, 
    random_state=300
)

In [79]:
# pipe = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
# pipe = make_pipeline(StandardScaler(with_mean=False), LassoCV(alphas=10e4))
pipe = make_pipeline(LassoCV(verbose=100))

In [78]:
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [ ]:
# aa = csr_matrix(X_train).tocsr()

In [ ]:
params = {'lassocv__n_alphas': [2]}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=True)
model.fit(X_train, y_train);

Fitting 1 folds for each of 1 candidates, totalling 1 fits
(array([-0.        , -0.05545308,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        , -0.        ,
    

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   33.6s finished


(array([ -0.       ,  -0.       ,   0.       ,  -0.       ,   0.       ,
        -0.       ,  -0.       ,  -0.       ,  -0.       ,  -0.       ,
        -0.       ,  -0.       ,  -0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,   0.       ,  -0.       ,  -0.       ,
        -0.       ,   0.       ,   0.       ,  -0.       ,   0.       ,
        -0.       ,  -0.       ,   0.       ,   0.       ,   0.       ,
         0.       ,   0.       ,  -0.       ,   0.       ,   0.       ,
        -0.       ,   0.       ,   0.       ,  -0.       ,  -0.       ,
        -0.       ,  -0.       ,  -0.       ,  -0.       ,  -0.       ,
        -0.       ,   0.       ,   0.       ,  -0.       ,  -0.       ,
        -0.       ,   0.       ,   0.       ,  -0.       ,   0.       ,
         0.       ,  -0.       ,  -0.       ,   0.       ,  -0.       ,
         0.       ,  -0.       ,   0.       ,   0.       ,  -0.

Are we in a high bias or high variance situation?

In [27]:
pd.DataFrame(model.cv_results_).T

,0
mean_fit_time,39.5206
mean_score_time,1.24574
mean_test_score,0.266576
mean_train_score,0.280809
params,{}
rank_test_score,1
split0_test_score,0.266576
split0_train_score,0.280809
std_fit_time,0
std_score_time,0


Let's look at the coefficients.

In [28]:
pd.Series(model.best_estimator_.steps[-1][-1].coef_).describe()

count     212.000000
mean      191.612297
std      1065.393105
min      -801.359330
25%      -338.115987
50%        -0.000000
75%       229.013180
max      7050.268467
dtype: float64

Which are the coefs that go to 0?

In [ ]:
X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]

In [30]:
model.best_estimator_.steps[-1][-1].alpha_

0.06388135132905054

In [71]:
model.best_estimator_.steps[-1][-1].alphas_

array([ 63.88135133,   0.06388135])

In [32]:
model.best_estimator_.steps[-1][-1].coef_.shape

(212,)

In [38]:
model.best_estimator_.steps[-1][-1].n_iter_

829